In [1]:
import os,sys
import pandas as pd
import pickle
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from wer import calculate_wer

from Models import cnn_rnn_model,simple_rnn,model_2,bidirectional_rnn_model_gpu,bidirectional_rnn_model
from Batch import Batch


In [ ]:
data=pd.read_json('../artifacts/meta.json')
data.drop(data[data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/features.pkl','rb') as tel:
    features=pickle.load(tel)
    tel.close()

In [ ]:
valid_data=pd.read_json("../artifacts/valid_meta.json")
valid_data.drop(valid_data[valid_data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/valid_features.pkl','rb') as val:
    valid_features=pickle.load(val)
    val.close()

In [ ]:
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()
MINI_BATCH_SIZE=250


audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [ ]:
bidirectional_rnn_gpu=bidirectional_rnn_model_gpu(input_dim=26, 
                        units=250)

In [ ]:
cnn_rnn = cnn_rnn_model(input_dim=26, # change to 13 if you would like to use MFCC features
                        filters=250,
                        kernel_size=4, 
                        conv_stride=1,
                        conv_border_mode='same',
                        units=200,output_dim=29)

In [ ]:
deep_speech = model_2(input_dim=26,
                filters=100,
                kernel_size=4, 
                conv_stride=1,
                conv_border_mode='valid',
                units=250,
                activation='tanh',
                dropout_rate=0.2,
                number_of_layers=2,
                output_dim=29)

In [ ]:
with open('models\bidirectional_rnn_gpu.h5', 'rb') as t:
        bidirectional_rnn_model_hist=pickle.load( t)

In [ ]:
with open('models\deep_speech.h5', 'rb') as f:
        deep_speech_hist=pickle.load( f)

In [ ]:
with open('models/simple_rnn_model.pickle', 'rb') as g:
        cnn_hist=pickle.load( g)

In [ ]:
deep_speech.load_weights(deep_speech_hist)

In [ ]:
bidirectional_rnn_model.load_weights(bidirectional_rnn_model_hist)

In [ ]:
simple_rnn_model.load_weights(simple_rnn_hist)

In [ ]:
predict(audio_gen,17,'validation',deep_speech)

In [ ]:
predict(audio_gen,17,'validation',deep_speech)

In [ ]:
predict(audio_gen,17,'validation',deep_speech)